In [1]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [2]:
import pandas as pd
import pyspark
from pyspark.sql import *

In [3]:
from pyspark.ml.feature  import Tokenizer,RegexTokenizer,StopWordsRemover,CountVectorizer,OneHotEncoderEstimator,StringIndexer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import LogisticRegression

In [4]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml.feature import Word2Vec

In [5]:
sc=pyspark.SparkContext()

In [6]:
spark= SparkSession \
       .builder \
       .appName("PA3") \
       .config("spark.some.config.option","some-value") \
       .getOrCreate() 

In [7]:
train_pd = pd.read_csv('train.csv')

In [8]:
mapping_pd = pd.read_csv('mapping.csv')


In [9]:
mapping=spark.createDataFrame(mapping_pd)


In [10]:
train=spark.createDataFrame(train_pd)

In [11]:
train.show()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [12]:
regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")


train=regexTokenizer.transform(train)

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

train=stopwordsRemover.transform(train)

word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="filtered", outputCol="features")
model3 = word2Vec.fit(train)

train = model3.transform(train)


In [13]:
mapping=spark.createDataFrame(mapping_pd,['id',"genre"])

In [14]:
mapping.show()
count=mapping.count()

+---+----------------+
| id|           genre|
+---+----------------+
|  0|           Drama|
|  1|          Comedy|
|  2|    Romance Film|
|  3|        Thriller|
|  4|          Action|
|  5|    World cinema|
|  6|   Crime Fiction|
|  7|          Horror|
|  8| Black-and-white|
|  9|           Indie|
| 10|Action/Adventure|
| 11|       Adventure|
| 12|     Family Film|
| 13|      Short Film|
| 14|  Romantic drama|
| 15|       Animation|
| 16|         Musical|
| 17| Science Fiction|
| 18|         Mystery|
| 19| Romantic comedy|
+---+----------------+



In [15]:

train.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[0.33213255401043...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[0.13228705924283...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|[0.27466382128406...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|[the, lemon, drop...|[lemon, drop, kid...|[0.26317628090281...|
|  595909|   A Cry in the Dark|Seventh-da

In [ ]:
import pyspark.sql.functions as f

In [ ]:
train.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[0.33213255401043...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[0.13228705924283...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|[0.27466382128406...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|[the, lemon, drop...|[lemon, drop, kid...|[0.26317628090281...|
|  595909|   A Cry in the Dark|Seventh-da

In [ ]:
mapgenre=mapping.select("genre","id").rdd.collectAsMap()

In [ ]:
from pyspark.sql.types import *
def fun1(g):
    x=[]
    print(g)
    for i in g[1:-1].split(","):
        x.append(mapgenre.get(i.strip()[1:-1]))
    x.sort()
    return x
udff=f.udf(fun1,ArrayType(IntegerType()))

train=train.withColumn("lab",udff("genre"))

In [ ]:
train.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features|             lab|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[0.33213255401043...|          [0, 5]|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[0.13228705924283...|  [0, 4, 10, 17]|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|[0.27466382128406...|      [0, 4, 16]|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|[th

In [ ]:
def fun2(g):
    y=1
    x=[]
    for i in range(0,count):
        x.append(0)
    
    
    for i in g:
        x[i]=1
        
    
    return x
udff=f.udf(fun2,ArrayType(IntegerType()))
traindata=train.withColumn("label",udff("lab"))
traindata.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               words|            filtered|            features|             lab|               label|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|[0.33213255401043...|          [0, 5]|[1, 0, 0, 0, 0, 1...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[the, nation, of,...|[nation, panem, c...|[0.13228705924283...|  [0, 4, 10, 17]|[1, 0, 0, 0, 1, 0...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, induch...|[0.2746638

In [ ]:
#Training the model and predicting on train data 
# x=[]
# for i in range(0,count):
    
#     def parsePoint(line):
    
#         return LabeledPoint(line.label[i], MLLibVectors.fromML(line.features))
#     parsedData = traindata.rdd.map(parsePoint)
#     model = LogisticRegressionWithLBFGS.train(parsedData)
#     labelsAndPreds = traindata.rdd.map(lambda p: (p.movie_id, model.predict(MLLibVectors.fromML(p.features))))
#     x.append(labelsAndPreds.collect())

In [ ]:
# y={}
# for i in x:
#     for j in i:
#         if j[0] in y:
#             y[j[0]].append(j[1])
#         else:
#             y[j[0]]=[]
#             y[j[0]].append(j[1])

In [ ]:
# for i in y:
#     print(i,y[i])

In [ ]:
test_pd = pd.read_csv('test.csv')
test=spark.createDataFrame(test_pd)
test.show()

+--------+--------------------+--------------------+
|movie_id|          movie_name|                plot|
+--------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|
|29062594|A la salida nos v...|A group of teenag...|
| 9252321|   Come Back, Africa|This story of a Z...|
|13455076|       A Merry Mixup|The Stooges play ...|
|24165951|        Getting Even|A soldier-of-fort...|
| 1925869|  River of No Return|Set in the Northw...|
|10799612|          Amici miei|Like in many othe...|
|28238240|Mickey's Big Game...|Mickey and the Sc...|
|17124781|The Good, the Bad...|In the desert wil...|
|28207941|    The Dancing Fool|Bimbo and Koko ar...|
|19174305|              Tahaan|Tahaan  lives wit...|
|18392317|     Mysterious Mose|Betty is startled...|
|34420857|Kelviyum Naane Pa...|Nirmal ([[Karthik...|
| 4039635|   First on the Moon|A group of journa...|
| 8034072|  Journey of a Woman|Vaibhavari Sahay,...|
| 4016437|     Sophie's Choice|In 1947, the mo

In [ ]:
from pyspark.ml.feature import Word2Vec

In [ ]:



test=regexTokenizer.transform(test)



test=stopwordsRemover.transform(test)





In [ ]:


testdata = model3.transform(test)

In [ ]:
testdata.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               words|            filtered|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|The film is based...|[the, film, is, b...|[film, based, eve...|[0.20447801023272...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|[0.35377794303573...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|[0.21313134880154...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|[0.28419763109449...|
|24165951|        Getting Even|A soldier-of-fort...|[a, soldier, of, ...|[soldier, fortune...|[0.15919354464858...|
| 1925869|  River of No Return|Set in the Northw...|[set, in, the, no...

In [ ]:
x=[]
for i in range(0,count):
    
    def parsePoint(line):
    
        return LabeledPoint(line.label[i], MLLibVectors.fromML(line.features))
    parsedData = traindata.rdd.map(parsePoint)
    model = LogisticRegressionWithLBFGS.train(parsedData)
    labelsAndPreds = testdata.rdd.map(lambda p: (p.movie_id, model.predict(MLLibVectors.fromML(p.features))))
    x.append(labelsAndPreds.collect())

In [ ]:
y={}
for i in x:
    for j in i:
        if j[0] in y:
            y[j[0]].append(j[1])
        else:
            y[j[0]]=[]
            y[j[0]].append(j[1])
for i in y:
    print(i,y[i])